In [2]:
# loading in and transforming data
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,ConcatDataset
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#from skimage import io, transform
from PIL import Image

# visualizing data
import matplotlib.pyplot as plt
import numpy as np
import warnings

# load dataset information
import yaml

# image writing
import imageio
from skimage import img_as_ubyte

# Clear GPU cache
torch.cuda.empty_cache()

In [3]:
# configuration
# WholeDatasetName = 'CVC-ClinicDB'
WholeDatasetName = 'UTTQ'

model_type = 'B0'
_model_name = 'ESFP_{}_Endo_{}'.format(model_type,WholeDatasetName)
config = open('Configure.yaml')
config = yaml.safe_load(config)

init_trainsize = 312
batch_size = 8

repeats = 1
n_epochs = 100
if_renew = True

In [4]:
class SplittingDataset(Dataset):
    """
    dataloader for polyp segmentation tasks
    """
    def __init__(self, image_root, gt_root):

        self.images = [image_root + f for f in os.listdir(image_root) if f.endswith('.jpeg') or f.endswith('.png') or f.endswith('.jpg')]
        # print(len(self.images))
        self.gts = [gt_root + f for f in os.listdir(gt_root) if f.endswith('.png') or f.endswith('.jpeg') or f.endswith('.jpg')]
        # print(len(self.gts))
        self.images = sorted(self.images)
        self.gts = sorted(self.gts)
        self.filter_files()
        self.size = len(self.images)
        self.transform = transforms.ToTensor()

    def __getitem__(self, index):
        
        image = self.rgb_loader(self.images[index])
        gt = self.binary_loader(self.gts[index])
        name = self.images[index].split('/')[-1]
        return self.transform(image), self.transform(gt), name

    def filter_files(self):
        assert len(self.images) == len(self.gts)
        images = []
        gts = []
        for img_path, gt_path in zip(self.images, self.gts):
            img = Image.open(img_path)
            gt = Image.open(gt_path)
            if img.size == gt.size:
                images.append(img_path)
                gts.append(gt_path)
        self.images = images
        self.gts = gts

    def rgb_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

    def binary_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('L')

    def __len__(self):
        return self.size

In [5]:
def splitDataset(renew, root_dir):
    
    split_train_images_save_path = './Endoscope-WL/{}_Splited/trainSplited/images'.format(WholeDatasetName)
    os.makedirs(split_train_images_save_path, exist_ok=True)
    split_train_masks_save_path = './Endoscope-WL/{}_Splited/trainSplited/masks'.format(WholeDatasetName)
    os.makedirs(split_train_masks_save_path, exist_ok=True)
    
    split_validation_images_save_path = './Endoscope-WL/{}_Splited/validationSplited/images'.format(WholeDatasetName)
    os.makedirs(split_validation_images_save_path, exist_ok=True)
    split_validation_masks_save_path = './Endoscope-WL/{}_Splited/validationSplited/masks'.format(WholeDatasetName)
    os.makedirs(split_validation_masks_save_path, exist_ok=True)
    
    split_test_images_save_path = './Endoscope-WL/{}_Splited/testSplited/images'.format(WholeDatasetName)
    os.makedirs(split_test_images_save_path, exist_ok=True)
    split_test_masks_save_path = './Endoscope-WL/{}_Splited/testSplited/masks'.format(WholeDatasetName)
    os.makedirs(split_test_masks_save_path, exist_ok=True)
    
    if renew == True:
    
        DatasetList = []
        
        # images_train_path = config['dataset']['train_' + str(WholeDatasetName) + '_dataset'] + '/images/'
        images_train_path = root_dir + "/train/images/"
        # masks_train_path = config['dataset']['train_' + str(WholeDatasetName) + '_dataset'] + '/masks/'
        masks_train_path = root_dir + "/train/mask_images/"
        Dataset_part_train = SplittingDataset(images_train_path, masks_train_path)
        DatasetList.append(Dataset_part_train)
         # images_test_path = config['dataset']['test_' + str(WholeDatasetName) + '_img']
        images_val_path = root_dir + "/valid/images/"
        # # masks_test_path = config['dataset']['test_' + str(WholeDatasetName) + '_label']
        masks_val_path = root_dir + "/valid/mask_images/"
        Dataset_part_val = SplittingDataset(images_val_path, masks_val_path)
        DatasetList.append(Dataset_part_val)

        # images_test_path = config['dataset']['test_' + str(WholeDatasetName) + '_img']
        images_test_path = root_dir + "/test/images/"
        # # masks_test_path = config['dataset']['test_' + str(WholeDatasetName) + '_label']
        masks_test_path = root_dir + "/test/mask_images/"
        Dataset_part_test = SplittingDataset(images_test_path, masks_test_path)
        DatasetList.append(Dataset_part_test)
                                    
        wholeDataset = ConcatDataset([DatasetList[0], DatasetList[1], DatasetList[2]])
        # val_num = int(0.1*len(wholeDataset))
        # test_num = int(0.1*len(wholeDataset))
                                    
        # train_num = len(wholeDataset) - val_num - test_num
        val_num = len(os.listdir(root_dir + "/valid/images/"))
        # print(val_num)
        # test_num = len(os.listdir(root_dir + "/new_test/images/"))
        train_num = len(os.listdir(root_dir + "/train/images/"))
        # print(train_num)
        
        # train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(wholeDataset, [train_num, val_num, test_num])
        train_dataset = torch.utils.data.Subset(wholeDataset, range(train_num))
        val_dataset = torch.utils.data.Subset(wholeDataset, range(train_num, train_num + val_num))
        test_dataset = torch.utils.data.Subset(wholeDataset, range(train_num + val_num, len(wholeDataset)))
                           
        train_loader = DataLoader(dataset=train_dataset,batch_size=1,shuffle=False)
        val_loader = DataLoader(dataset=val_dataset,batch_size=1,shuffle=False)
        test_loader = DataLoader(dataset=test_dataset,batch_size=1,shuffle=False)
    
        iter_train = iter(train_loader)
        for i in range(len(train_loader)):
            image, gt, name = iter_train.next()
            image_data = image.data.cpu().numpy().squeeze().transpose(1,2,0)
            gt_data = gt.data.cpu().numpy().squeeze()
            imageio.imwrite(split_train_images_save_path + '/' + name[0],img_as_ubyte(image_data))
            imageio.imwrite(split_train_masks_save_path + '/' + name[0],img_as_ubyte(gt_data))
            
        iter_val = iter(val_loader)
        for i in range(len(val_loader)):
            image, gt, name = iter_val.next()
            image_data = image.data.cpu().numpy().squeeze().transpose(1,2,0)
            gt_data = gt.data.cpu().numpy().squeeze()
            imageio.imwrite(split_validation_images_save_path + '/' + name[0],img_as_ubyte(image_data))
            imageio.imwrite(split_validation_masks_save_path + '/' + name[0],img_as_ubyte(gt_data))
            
        
        iter_test = iter(test_loader)
        for i in range(len(test_loader)):
            image, gt, name = iter_test.next()
            image_data = image.data.cpu().numpy().squeeze().transpose(1,2,0)
            gt_data = gt.data.cpu().numpy().squeeze()
            imageio.imwrite(split_test_images_save_path + '/' + name[0],img_as_ubyte(image_data))
            imageio.imwrite(split_test_masks_save_path + '/' + name[0],img_as_ubyte(gt_data))        
    
    return split_train_images_save_path, split_train_masks_save_path, split_validation_images_save_path, split_validation_masks_save_path, split_test_images_save_path, split_test_masks_save_path

In [6]:
if_renew = False
root_dir = "/home/baoanh/baoanh/DATN/Ungthuthucquan"
train_images_path, train_masks_path, val_images_path, val_masks_path, test_images_path, test_masks_path = splitDataset(if_renew, root_dir)

In [7]:
# root_dir = "/home/baoanh/baoanh/DATN/dataset/Viem_daday_hp_am"
# train_images_path = root_dir + "/new_train/images"
# train_masks_path = root_dir + "/new_train/mask_images"
# val_images_path = root_dir + "/new_valid/images"
# val_masks_path = root_dir + "/new_valid/mask_images"
# test_images_path = root_dir + "/new_test/images"
# test_masks_path = root_dir + "/new_test/mask_images"

In [8]:
class PolypDataset(Dataset):
    """
    dataloader for polyp segmentation tasks
    """
    def __init__(self, image_root, gt_root, trainsize, augmentations):
        self.trainsize = trainsize
        self.augmentations = augmentations
        # print(self.augmentations)
        self.images = [image_root + f for f in os.listdir(image_root) if f.endswith('.jpeg') or f.endswith('.png') or f.endswith('.jpg')]
        #print(image_root)
        self.gts = [gt_root + f for f in os.listdir(gt_root) if f.endswith('.png') or f.endswith('.jpeg') or f.endswith('.jpg')]
        self.images = sorted(self.images)
        self.gts = sorted(self.gts)
        self.filter_files()
        self.size = len(self.images)
        if self.augmentations == True:
            print('Using RandomRotation, RandomFlip')
            self.img_transform = transforms.Compose([
                transforms.RandomRotation(90, resample=False, expand=False, center=None),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.Resize((self.trainsize, self.trainsize)),
                transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0, hue=0),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])])
            self.gt_transform = transforms.Compose([
                transforms.RandomRotation(90, resample=False, expand=False, center=None),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.Resize((self.trainsize, self.trainsize)),
                transforms.ToTensor()])
            
        else:
            print('no augmentation')
            self.img_transform = transforms.Compose([
                transforms.Resize((self.trainsize, self.trainsize)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])])
            
            self.gt_transform = transforms.Compose([
                transforms.Resize((self.trainsize, self.trainsize)),
                transforms.ToTensor()])
            

    def __getitem__(self, index):
        
        image = self.rgb_loader(self.images[index])
        gt = self.binary_loader(self.gts[index])
        
        seed = np.random.randint(2147483647) # make a seed with numpy generator 
        np.random.seed(seed) # apply this seed to img tranfsorms
        torch.manual_seed(seed) # needed for torchvision 0.7
        if self.img_transform is not None:
            image = self.img_transform(image)
            
        np.random.seed(seed) # apply this seed to img tranfsorms
        torch.manual_seed(seed) # needed for torchvision 0.7
        if self.gt_transform is not None:
            gt = self.gt_transform(gt)
        return image, gt

    def filter_files(self):
        assert len(self.images) == len(self.gts)
        images = []
        gts = []
        for img_path, gt_path in zip(self.images, self.gts):
            img = Image.open(img_path)
            gt = Image.open(gt_path)
            if img.size == gt.size:
                images.append(img_path)
                gts.append(gt_path)
        self.images = images
        self.gts = gts

    def rgb_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

    def binary_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            # return img.convert('1')
            return img.convert('L')

    def resize(self, img, gt):
        assert img.size == gt.size
        w, h = img.size
        if h < self.trainsize or w < self.trainsize:
            h = max(h, self.trainsize)
            w = max(w, self.trainsize)
            return img.resize((w, h), Image.BILINEAR), gt.resize((w, h), Image.NEAREST)
        else:
            return img, gt

    def __len__(self):
        return self.size

class test_dataset:
    def __init__(self, image_root, gt_root, testsize):
        self.testsize = testsize
        self.images = [image_root + f for f in os.listdir(image_root) if f.endswith('.jpeg') or f.endswith('.png') or f.endswith('.jpg')]
        self.gts = [gt_root + f for f in os.listdir(gt_root) if f.endswith('.tif') or f.endswith('.png') or f.endswith('.jpeg') or f.endswith('.jpg')]
        self.images = sorted(self.images)
        self.gts = sorted(self.gts)
        self.transform = transforms.Compose([
            transforms.Resize((self.testsize, self.testsize)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])])
        self.gt_transform = transforms.ToTensor()
        self.size = len(self.images)
        self.index = 0

    def load_data(self):
        image = self.rgb_loader(self.images[self.index])
        image = self.transform(image).unsqueeze(0)
        gt = self.binary_loader(self.gts[self.index])
        name = self.images[self.index].split('/')[-1]
        if name.endswith('.jpeg'):
            name = name.split('.jpeg')[0] + '.png'
        self.index += 1
        return image, gt, name

    def rgb_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

    def binary_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('L')

In [9]:
from Encoder import mit
from Decoder import mlp
from mmcv.cnn import ConvModule

class ESFPNetStructure(nn.Module):

    def __init__(self, embedding_dim = 160):
        super(ESFPNetStructure, self).__init__()
        
        # Backbone
        if model_type == 'B0':
            self.backbone = mit.mit_b0()
        if model_type == 'B1':
            self.backbone = mit.mit_b1()
        if model_type == 'B2':
            self.backbone = mit.mit_b2()
        if model_type == 'B3':
            self.backbone = mit.mit_b3()
        if model_type == 'B4':
            self.backbone = mit.mit_b4()
        if model_type == 'B5':
            self.backbone = mit.mit_b5()
        
        self._init_weights()  # load pretrain
        
        # LP Header
        self.LP_1 = mlp.LP(input_dim = self.backbone.embed_dims[0], embed_dim = self.backbone.embed_dims[0])
        self.LP_2 = mlp.LP(input_dim = self.backbone.embed_dims[1], embed_dim = self.backbone.embed_dims[1])
        self.LP_3 = mlp.LP(input_dim = self.backbone.embed_dims[2], embed_dim = self.backbone.embed_dims[2])
        self.LP_4 = mlp.LP(input_dim = self.backbone.embed_dims[3], embed_dim = self.backbone.embed_dims[3])
        
        # Linear Fuse
        self.linear_fuse34 = ConvModule(in_channels=(self.backbone.embed_dims[2] + self.backbone.embed_dims[3]), out_channels=self.backbone.embed_dims[2], kernel_size=1,norm_cfg=dict(type='BN', requires_grad=True))
        self.linear_fuse23 = ConvModule(in_channels=(self.backbone.embed_dims[1] + self.backbone.embed_dims[2]), out_channels=self.backbone.embed_dims[1], kernel_size=1,norm_cfg=dict(type='BN', requires_grad=True))
        self.linear_fuse12 = ConvModule(in_channels=(self.backbone.embed_dims[0] + self.backbone.embed_dims[1]), out_channels=self.backbone.embed_dims[0], kernel_size=1,norm_cfg=dict(type='BN', requires_grad=True))
        
        # Fused LP Header
        self.LP_12 = mlp.LP(input_dim = self.backbone.embed_dims[0], embed_dim = self.backbone.embed_dims[0])
        self.LP_23 = mlp.LP(input_dim = self.backbone.embed_dims[1], embed_dim = self.backbone.embed_dims[1])
        self.LP_34 = mlp.LP(input_dim = self.backbone.embed_dims[2], embed_dim = self.backbone.embed_dims[2])
        
        # Final Linear Prediction
        self.linear_pred = nn.Conv2d((self.backbone.embed_dims[0] + self.backbone.embed_dims[1] + self.backbone.embed_dims[2] + self.backbone.embed_dims[3]), 1, kernel_size=1)
        
    def _init_weights(self):
        
        if model_type == 'B0':
            pretrained_dict = torch.load('./Pretrained/mit_b0.pth')
        if model_type == 'B1':
            pretrained_dict = torch.load('./Pretrained/mit_b1.pth')
        if model_type == 'B2':
            pretrained_dict = torch.load('./Pretrained/mit_b2.pth')
        if model_type == 'B3':
            pretrained_dict = torch.load('./Pretrained/mit_b3.pth')
        if model_type == 'B4':
            pretrained_dict = torch.load('./Pretrained/mit_b4.pth')
        if model_type == 'B5':
            pretrained_dict = torch.load('./Pretrained/mit_b5.pth')
            
            
        model_dict = self.backbone.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.backbone.load_state_dict(model_dict)
        print("successfully loaded!!!!")
        
        
    def forward(self, x):
        
        ##################  Go through backbone ###################
        
        B = x.shape[0]
        
        #stage 1
        out_1, H, W = self.backbone.patch_embed1(x)
        for i, blk in enumerate(self.backbone.block1):
            out_1 = blk(out_1, H, W)
        out_1 = self.backbone.norm1(out_1)
        out_1 = out_1.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()  #(Batch_Size, self.backbone.embed_dims[0], 88, 88)
        
        # stage 2
        out_2, H, W = self.backbone.patch_embed2(out_1)
        for i, blk in enumerate(self.backbone.block2):
            out_2 = blk(out_2, H, W)
        out_2 = self.backbone.norm2(out_2)
        out_2 = out_2.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()  #(Batch_Size, self.backbone.embed_dims[1], 44, 44)
        
        # stage 3
        out_3, H, W = self.backbone.patch_embed3(out_2)
        for i, blk in enumerate(self.backbone.block3):
            out_3 = blk(out_3, H, W)
        out_3 = self.backbone.norm3(out_3)
        out_3 = out_3.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()  #(Batch_Size, self.backbone.embed_dims[2], 22, 22)
        
        # stage 4
        out_4, H, W = self.backbone.patch_embed4(out_3)
        for i, blk in enumerate(self.backbone.block4):
            out_4 = blk(out_4, H, W)
        out_4 = self.backbone.norm4(out_4)
        out_4 = out_4.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()  #(Batch_Size, self.backbone.embed_dims[3], 11, 11)
        
        # go through LP Header
        lp_1 = self.LP_1(out_1)
        lp_2 = self.LP_2(out_2)  
        lp_3 = self.LP_3(out_3)  
        lp_4 = self.LP_4(out_4)
        
        # linear fuse and go pass LP Header
        lp_34 = self.LP_34(self.linear_fuse34(torch.cat([lp_3, F.interpolate(lp_4,scale_factor=2,mode='bilinear', align_corners=False)], dim=1)))
        lp_23 = self.LP_23(self.linear_fuse23(torch.cat([lp_2, F.interpolate(lp_34,scale_factor=2,mode='bilinear', align_corners=False)], dim=1)))
        lp_12 = self.LP_12(self.linear_fuse12(torch.cat([lp_1, F.interpolate(lp_23,scale_factor=2,mode='bilinear', align_corners=False)], dim=1)))
        
        # get the final output
        lp4_resized = F.interpolate(lp_4,scale_factor=8,mode='bilinear', align_corners=False)
        lp3_resized = F.interpolate(lp_34,scale_factor=4,mode='bilinear', align_corners=False)
        lp2_resized = F.interpolate(lp_23,scale_factor=2,mode='bilinear', align_corners=False)
        lp1_resized = lp_12
        
        out = self.linear_pred(torch.cat([lp1_resized, lp2_resized, lp3_resized, lp4_resized], dim=1))
        
        return out

In [10]:
import torch
import cv2
import numpy as np
import torch
from scipy.ndimage import distance_transform_edt

# def calc_dist_map(seg):
#     res = np.zeros_like(seg)
#     posmask = seg.astype(np.bool)

#     if posmask.any():
#         negmask = ~posmask
#         res = distance_transform_edt(negmask) * negmask - (distance_transform_edt(posmask) - 1) * posmask

#     return res.astype(np.float32)

# def calc_dist_map_batch(y_true):
#     y_true_numpy = y_true.cpu().detach().numpy()
#     return np.array([calc_dist_map(y) for y in y_true_numpy])

# def surface_loss_torch(y_pred, y_true):
#     import torch
#     import torch.nn.functional as F
#     y_true_dist_map = torch.from_numpy(calc_dist_map_batch(y_true)).to(y_pred.device)
#     multipled = y_pred * y_true_dist_map
#     return torch.mean(multipled)
import torch
import torch.nn.functional as F

def focal_loss(pred, target, gamma=2, alpha=0.25, smooth=0.0001):
    num = target.size(0)
    pred = pred.contiguous()
    target = target.contiguous()

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    dice_coefficient = (2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)

    # Calculate the focal weights for each pixel
    focal_weights = torch.pow(1 - dice_coefficient, gamma)

    # Calculate the focal loss (element-wise multiplication with alpha)
    focal_loss = -alpha * torch.log(dice_coefficient) * focal_weights

    return focal_loss.sum() / num
def ange_structure_loss(pred, mask, smooth=1):
    
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=15, stride=1, padding=7) - mask)
    wbce = F.binary_cross_entropy_with_logits(pred, mask, reduction='mean')
    wbce = (weit*wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask)*weit).sum(dim=(2, 3))
    union = ((pred + mask)*weit).sum(dim=(2, 3))
    wiou = 1 - (inter + smooth)/(union - inter + smooth)
    wfocal = focal_loss(pred, mask) 
    wfocal = (weit*wfocal).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))
    return (wbce + wiou + wfocal).mean()

def dice_loss_coff(pred, target, smooth = 0.0001):
    
    num = target.size(0)
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)
    
    return loss.sum()/num

In [11]:
from torch.autograd import Variable
def evaluate():  
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    ESFPNet.eval()
    
    val = 0
    count = 0

    smooth = 1e-4
    
    val_loader = test_dataset(val_images_path + '/',val_masks_path + '/', init_trainsize)
    for i in range(val_loader.size):
        image, gt, name = val_loader.load_data()
        gt = np.asarray(gt, np.float32)
        gt /= (gt.max() + 1e-8)

        image = image.cuda()
        
        pred= ESFPNet(image)
        pred = F.upsample(pred, size=gt.shape, mode='bilinear', align_corners=False)
        pred = pred.sigmoid()
        threshold = torch.tensor([0.5]).to(device)
        pred = (pred > threshold).float() * 1

        pred = pred.data.cpu().numpy().squeeze()
        pred = (pred - pred.min()) / (pred.max() - pred.min() + 1e-8)
        
        target = np.array(gt)
        
        input_flat = np.reshape(pred,(-1))
        target_flat = np.reshape(target,(-1))
 
        intersection = (input_flat*target_flat)
        
        loss =  (2 * intersection.sum() + smooth) / (pred.sum() + target.sum() + smooth)

        a =  '{:.4f}'.format(loss)
        a = float(a)
        
        val = val + a
        count = count + 1
        
    ESFPNet.train()
    
    return val/count, loss

In [12]:
# train the network
def load_model(model, model_path):
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f"Loaded pre-trained model from {model_path}")
    else:
        print("No pre-trained model found. Starting from scratch.")
def training_loop(n_epochs, model, ESFPNet_optimizer, numIters):
    
    # keep track of losses over time
    losses = []
    val_losses = []
    coeff_max = 0
    
    # set up data and then train
    trainDataset = PolypDataset(train_images_path + '/', train_masks_path + '/', trainsize=init_trainsize, augmentations = True)
    train_loader = DataLoader(dataset=trainDataset,batch_size=batch_size,shuffle=True)
    
    iter_X = iter(train_loader)
    steps_per_epoch = len(iter_X)
    num_epoch = 0
    total_steps = (n_epochs+1)*steps_per_epoch
    num_to_stop = 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    for step in range(1, total_steps):

        # Reset iterators for each epoch
        if step % steps_per_epoch == 0:
            iter_X = iter(train_loader)
            num_epoch = num_epoch + 1
        
        # make sure to scale to a range -1 to 1
        images, masks = iter_X.next()
        
        # move images to GPU if available (otherwise stay on CPU)
        images = images.to(device)
        masks = masks.to(device)


        # ============================================
        #            TRAIN THE NETWORKS
        # ============================================
       
        ESFPNet_optimizer.zero_grad()
        
        # 1. Compute the losses from the network
        
        out = ESFPNet(images)
        out = F.interpolate(out, scale_factor=4, mode='bilinear', align_corners=False)
        
        loss = ange_structure_loss(out, masks)
        # boundary_loss = DistBinaryDiceLoss()
        # loss = boundary_loss(out, masks)
        # print(out.shape, masks.shape)
        # l1_reg = torch.tensor(0.0).to(device)
        l2_reg = torch.tensor(0.0).to(device)
        for module in model.modules():
            mask = None
            weight = None
            for name, buffer in module.named_buffers():
                if name == "weight_mask":
                    mask = buffer
            for name, param in module.named_parameters():
                if name == "weight_orig":
                    weight = param
            if mask is not None and weight is not None:
                # l1_reg += torch.norm(mask * weight, 1)
                l2_reg += 0.5 * torch.norm(mask * weight, 2) 
        l1_regularization_strength = 1e-3
        loss += l1_regularization_strength * l2_reg
        loss.backward()
        ESFPNet_optimizer.step() 
        
        # ============================================
        #            TRAIN THE NETWORKS
        # ============================================
        # Print the log info
        if step % steps_per_epoch == 0:
            save_model_path = './SaveModel/{}_LA_{:1d}'.format(_model_name,numIters)
            os.makedirs(save_model_path, exist_ok=True)
            losses.append(loss.item())
            print('Epoch [{:5d}/{:5d}] | preliminary loss: {:6.6f} '.format(num_epoch, n_epochs, loss.item()))
            # torch.save(ESFPNet, save_model_path + f'/ESFPNet_{num_epoch}.pt')
        if step % steps_per_epoch == 0:
            
            validation_coeff, val_loss = evaluate()
            val_losses.append(val_loss.item())
            print('Epoch [{:5d}/{:5d}] | validation_coeffient: {:6.6f} '.format(
                    num_epoch, n_epochs, validation_coeff))
            
            if coeff_max < validation_coeff:
                coeff_max = validation_coeff
                save_model_path = './SaveModel/{}_LA_{:1d}'.format(_model_name,numIters)
                os.makedirs(save_model_path, exist_ok=True)
                # print(save_model_path)
                torch.save(ESFPNet, save_model_path + '/ESFPNet.pt')
                print('Save Learning Ability Optimized Model at Epoch [{:5d}/{:5d}]'.format(num_epoch, n_epochs))
                num_to_stop = 0
            else:
                num_to_stop = num_to_stop + 1
    
        if num_to_stop >=10:
            break
                
    return losses,val_losses, coeff_max, num_epoch

In [13]:
def saveResult(numIters):
    
    save_path = './results/{}_LA_{:1d}/{}_Splited/'.format(_model_name,numIters,str(WholeDatasetName))
    os.makedirs(save_path, exist_ok=True)
    print(save_path)
        
    model_path =  './SaveModel/{}_LA_{:1d}'.format(_model_name,numIters)
    ESFPNetBest = torch.load(model_path + '/ESFPNet.pt')
    ESFPNetBest.eval()
    
    test_loader = test_dataset(test_images_path + '/', test_masks_path + '/', init_trainsize)
    for i in range(test_loader.size):
        image, gt, name = test_loader.load_data()
        gt = np.asarray(gt, np.float32)
        gt /= (gt.max() + 1e-8)
        image = image.cuda()

        pred = ESFPNetBest(image)
        pred = F.upsample(pred, size=gt.shape, mode='bilinear', align_corners=False)
        pred = pred.sigmoid()
        threshold = torch.tensor([0.5]).to(device)
        pred = (pred > threshold).float() * 1
        pred = pred.data.cpu().numpy().squeeze()
        pred = (pred - pred.min()) / (pred.max() - pred.min() + 1e-8)
        

        imageio.imwrite(save_path+name,img_as_ubyte(pred))

In [14]:
import torch.optim as optim

for i in range(repeats):
    
    # Clear GPU cache
    torch.cuda.empty_cache()
    
    ESFPNet = ESFPNetStructure()
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        ESFPNet.to(device)
        print('Models moved to GPU.')
    else:
        print('Only CPU available.')
    print('#####################################################################################')  
        
    # hyperparams for Adam optimizer
    lr=0.0001 #0.0001

    ESFPNet_optimizer = optim.AdamW(ESFPNet.parameters(), lr=lr)

    losses, val_losses, coeff_max, num_epoch = training_loop(n_epochs,ESFPNet, ESFPNet_optimizer, i+1)
    plt.plot(range(1, num_epoch+1), losses, color='b', label='Training Loss')
    plt.plot(range(1, num_epoch+1), val_losses, color='r', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    print('#####################################################################################')  
    print('optimize_m_dice: {:6.6f}'.format(coeff_max))
    plt.savefig(f"visualize_{str(WholeDatasetName)}.png")
    saveResult(i+1)
    print('#####################################################################################')  
    print('saved the results')
    print('#####################################################################################')  

successfully loaded!!!!
Models moved to GPU.
#####################################################################################
Using RandomRotation, RandomFlip


/home/baoanh/miniconda3/envs/esfp/lib/python3.10/site-packages/torchvision/transforms/transforms.py:1292: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed 0.14. Please use 'interpolation' instead.
  warnings.warn(


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 39 but got size 40 for tensor number 1 in the list.

In [ ]:
import os
import cv2
import glob

def specificity_score(y_true, y_pred):
    tn = sum((y_true == 0) & (y_pred == 0))
    fp = sum((y_true == 0) & (y_pred == 1))
    specificity = tn / (tn + fp)
    return  specificity
# Folder paths for masks and predicted masks
# Paths to folders containing masks and predicted images
mask_folder = "/home/baoanh/baoanh/DATN/dataset/Ungthuthucquan/test/mask_images/*.png"
predicted_mask_folder = "/home/baoanh/baoanh/DATN/ESFPNet/results/ESFP_B0_Endo_UTTQ_LA_1/UTTQ_Splited/*.jpeg"
# mask_folder = f"/home/baoanh/baoanh/DATN/ESFPNet/Endoscope-WL/{WholeDatasetName}_Splited/testSplited/masks/*.jpeg"
# mask_folder = "/home/baoanh/baoanh/DATN/dataset/Viem_daday_hp_am/new_test/mask_images/*.png"
# predicted_mask_folder = f"/home/baoanh/baoanh/DATN/ESFPNet/results/ESFP_B0_Endo_{WholeDatasetName}_LA_1/{WholeDatasetName}_Splited/*.png"
from skimage.io import imread
from skimage.transform import resize
mask_images = sorted(glob.glob(mask_folder))
predicted_mask_images = sorted(glob.glob(predicted_mask_folder))
print(len(mask_images), len(predicted_mask_images))

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score, f1_score, precision_score, recall_score
for i in range(len(mask_images)):
    pred = np.ndarray.flatten(imread(predicted_mask_images[i]) / 255) > 0.5
    gt = np.ndarray.flatten(imread(mask_images[i])) 
    # print(imread(predicted_mask_images[i]).shape)
    # print(imread(mask_images[i])[:,:,0].shape)
    # print(np.unique(imread(mask_images[i], 0)))
    # print(np.unique(imread(predicted_mask_images[i])))
    dice = []
    IoU = []
    precision = []
    recall = []
    specificity = []
    dice.append(f1_score(gt, pred))
    IoU.append(jaccard_score(gt, pred))
    precision.append(precision_score(gt, pred))
    recall.append(recall_score(gt, pred))
    specificity.append(specificity_score(gt, pred))
    # break
print(  np.mean(dice),
        np.mean(IoU),
        np.mean(precision),
        np.mean(recall),
        np.mean(specificity),
    )

55 55
0.8950418053826019 0.8100232296050968 0.9334296296787103 0.8596867009623541 0.9349839190841966


In [ ]:
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from PIL import Image
from sklearn.metrics import jaccard_score, f1_score, confusion_matrix
from skimage import img_as_bool
from PIL import Image
# Paths to folders containing masks and predicted images
mask_folder = "/home/baoanh/baoanh/DATN/ESFPNet/Endoscope-WL/UTTQ_Splited/testSplited/masks"
predicted_folder = "/home/baoanh/baoanh/DATN/ESFPNet/results/ESFP_B0_Endo_UTTQ_LA_1/UTTQ_Splited"

# List files in the folders
mask_files = sorted(os.listdir(mask_folder))
predicted_files = sorted(os.listdir(predicted_folder))

# Initialize lists to store true and predicted values
true_values = []
predicted_values = []

# Load and process masks and predicted images
for mask_file, pred_file in zip(mask_files, predicted_files):
    mask_path = os.path.join(mask_folder, mask_file)
    pred_path = os.path.join(predicted_folder, pred_file)
    
    # Load images as arrays (grayscale images)
    mask = np.array(Image.open(mask_path))
    pred = np.array(Image.open(pred_path))
    # print(pred.shape)
    # print(mask.shape)
    # Flatten images into 1D arrays
    true_values.extend(mask.flatten())
    predicted_values.extend(pred.flatten())
    # break

# Convert lists to NumPy arrays
true_values = np.array(true_values)/255 > 0.5
predicted_values = np.array(predicted_values)/255 > 0.5
print(np.unique(true_values))
print(np.unique(predicted_values))
# Calculate metrics
precision = precision_score(true_values, predicted_values)
recall = recall_score(true_values, predicted_values)
f1 = f1_score(true_values, predicted_values)
conf_matrix = confusion_matrix(true_values, predicted_values)

# Calculate mIoU (Jaccard Index)
jaccard_indices = jaccard_score(true_values, predicted_values, average=None)
miou = np.mean(jaccard_indices)

# Calculate mDice (Dice Coefficient)
def dice_coef(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    total = np.sum(y_true) + np.sum(y_pred)
    return (2.0 * intersection) / (total + 1e-6)

mdice = dice_coef(true_values, predicted_values)

# Calculate mSpecificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp + 1e-6)

mspecificity = specificity_score(true_values, predicted_values)
print("mDice:", mdice)
print("mIoU:", miou)
print("Recall:", recall)
print("Precision:", precision)
print("mSpecificity:", mspecificity)


[False  True]
[False  True]
mDice: 0.8657421821943052
mIoU: 0.8333686523165287
Recall: 0.8295869252187139
Precision: 0.9051924994991245
mSpecificity: 0.9651761249274172
